# Projecto 1 

### Integrantes

* Jonathan Rivera
* Julian Castro  
* Alejandro Gomez

### Caso

El Ministerio de Comercio, Industria y Turismo de Colombia, la Asociación Hotelera y
Turística de Colombia – COTELCO, cadenas hoteleras de la talla de Hilton, Hoteles Estelar,
Holiday Inn y hoteles pequeños ubicados en diferentes municipios de Colombia están
interesados en analizar las características de sitios turísticos que los hacen atractivos para
turistas locales o de otros países, ya sea para ir a conocerlos o recomendarlos. De igual
manera, quieren comparar las características de dichos sitios, con aquellos que han
obtenido bajas recomendaciones y que están afectando el número de turistas que llegan a
ellos. Adicionalmente, quieren tener un mecanismo para determinar la calificación que
tendrá un sitio por parte de los turistas y así, por ejemplo, aplicar estrategias para identificar
oportunidades de mejora que permitan aumentar la popularidad de los sitios y fomentar el
turismo.
Esos actores de turismo prepararon dos conjuntos de datos con reseñas de sitios turísticos.
Cada reseña tiene una calificación según el sentimiento que tuvo el turista al visitarlo. Estos
actores quieren lograr un análisis independiente de los conjuntos de datos y al final del
proyecto discutir sobre los grupos de científicos de datos e ingenieros de datos que
acompañarán el desarrollo real de este proyecto.

### Objetivos

Plantear un modelo de prediccion de calificaiciones de reseñas, basado en el procesamiento de textos, con un algoritmo que identifique la relacion semantica entre palabras y las calificaciones numericas

In [21]:
%pip install nltk
%pip install openpyxl
%pip install contractions
%pip install inflect
%pip install spacy
%pip install gensim
%pip install scikit-optimize
!python -m spacy download es_core_news_md
%pip install inflect
%pip install scikit-plot
%pip install spacy-language-detection
%pip install -U pip setuptools wheel
%pip install -U spacy
!python -m spacy download en_core_web_sm
! pip install langdetect


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\JHONATAN RIVERA\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



  Using cached langdetect-1.0.9-py3-none-any.whl


### Instalacion de librerias

In [13]:
import nltk
# Punkt permite separar un texto en frases.
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download("averaged_perceptron_tagger") 


[nltk_data] Downloading package punkt to C:\Users\JHONATAN
[nltk_data]     RIVERA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\JHONATAN
[nltk_data]     RIVERA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\JHONATAN
[nltk_data]     RIVERA\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\JHONATAN RIVERA\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [2]:
import pandas as pd
import numpy as np
import sys
!{sys.executable} -m pip install pandas-profiling

import re, string, unicodedata
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix


from sklearn.base import BaseEstimator, ClassifierMixin

import matplotlib.pyplot as plt

"c:\Users\JHONATAN" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


## 1. Perfilamiento y entendimiento de los datos

In [3]:
data=pd.read_csv('tipo2_entrenamiento_estudiantes.csv', sep=',', encoding = 'utf-8')
# Asignación a una nueva variable de los datos leidos
data_t=data

In [6]:
data.head()

,Review,Class
0,Muy buena atención y aclaración de dudas por p...,5
1,Buen hotel si están obligados a estar cerca de...,3
2,"Es un lugar muy lindo para fotografías, visite...",5
3,Abusados con la factura de alimentos siempre s...,3
4,Tuvimos un par de personas en el grupo que rea...,3


Los datos que tenemos se presentan de una manera muy simple, una columna con las reseñas en lenguaje natural, y otra columna con la calificación numerica de las reseñas

### 1.1 Lenguaje de los datos

Para el planteamiento de un modelo que analice textos, es esencial primero conocer el idioma en el que se encuentran los textos, esto porque tanto para la transformación de los textos (lematización, tokenización) para poder ser procesado por el modelo, como para configurar el modelo, es necesario que se tenga un lenguaje comun para todos los tokens

In [4]:
#Podemos usar la libreria langdetect para conocer el idioma de todos los textos, y podemos aprovecharnos de las funcionalidades 
# de pandas para facilmente agregar una columna con el idioma del texto para su posterior analizis
from langdetect import detect
data_t['idioma'] = data_t['Review'].apply(detect)
data_t

,Review,Class,idioma
0,Muy buena atención y aclaración de dudas por p...,5,es
1,Buen hotel si están obligados a estar cerca de...,3,es
2,"Es un lugar muy lindo para fotografías, visite...",5,es
3,Abusados con la factura de alimentos siempre s...,3,es
4,Tuvimos un par de personas en el grupo que rea...,3,es
...,...,...,...
7870,"Me parece buen sistema, agiliza el transporte,...",4,es
7871,"Fue una escapada de un día desde el complejo, ...",4,es
7872,La Plaza de la Revolución es un lugar emblemát...,3,es
7873,Es la segunda ocasión que me quedo en los cuar...,1,es


In [5]:
#Podemos realizar el conteo de los valores en la columna idioma para entender la proporcion linguistica de las reseñas
data_t['idioma'].value_counts()

idioma
es    7867
en       3
pt       2
it       2
sq       1
Name: count, dtype: int64

Como podemos virtualmente todas las reseñas se encuentran en español, con tan solo 8 reseñas de 7874 perteneciendo a otro idioma.
Como mencionabamos previamente, el modelo tiene que plantarse para un solo idioma, por lo que sera necesario remover las reseñas de otros idiomas en la fase de transformación de los datos

### 1.2 Entendimiento de los ratings

Entender y analizar la distribución de los datos de los rating puede ayudarnos a contribuir cualitativamente los objetivos de negocios

In [6]:
data_t.describe()

,Class
count,7875.000000
mean,3.502603
std,1.320435
min,1.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,5.000000


A partir de las estadisticas de los ratings de las reseñas podemos entender lo siguiente:
* No hay ratings invalidos dado que el maximo y el minimo se encuentran en una escala logica y consistente (1-5)
* En promedio las reseñas tienen una calificación de 3.5, por lo que puede ser evidencia de que se tiene una distribución apropiada de calificaciones (no estan sesgadas a ningun extremo).
* Lo anterior se puede confirmar observando la distribución de los datos en los cuartiles. El 25% de los ratings tienen una calificación maxima de 3, el 50% de 4, y el 75% de 5.
* Tenemos 7875 reseñas para la creación del modelo y su posterior validación. 

### 1.3 Calidad de los datos

#### 1.3.1 Completitud

In [7]:
((data_t.isnull().sum()/data_t.shape[0])).sort_values(ascending=False)

Review    0.0
Class     0.0
idioma    0.0
dtype: float64

Podemos observar que no tenemos valores nulos en nuestros datos, por lo tanto no sera necesario hacer un analizis de manejo de nulalidad

#### 1.3.2 Unicidad

In [8]:
data_t.duplicated(keep = False).sum()

102

Podemos observar que tenemos 102 registros duplicados, debido a que los duplicados no añaden valor alguno al modelo, pero si pueden introducir un sesgo (debido a que ciertas palabras pueden asosiarse de manera mas fuerte a un rating de manera erronea), tendremos que eliminarlos en la fase de limpieza de datos.

#### 1.3.3 Consistencia

Como veiamos previamente en el entendimiento de los ratings, estos datos son consistentes dado que todas las calificaciones se encuentran en un rango coherente y logico de (1-5)

#### 1.3.4 Validez

Debido a que no existe una relación cuantificable entre las columnas, no tiene cabida un analisis de validez de las relaciones entre columnas

### 2. Preparación de los datos

#### 2.1 Limpieza de los datos 

#### 2.1.1 Quitar duplicados

In [10]:
data_t= data_t.drop_duplicates()
data_t.duplicated(keep = False).sum()

0

Como mencionamos previamente, tenemos que deshacernos de los duplicados antes de alimentar el modelo con los datos. Observamos que el dataframe queda sin datos duplicados

#### 2.1.2 Quitar reseñas en otros idiomas

In [16]:
data_t= data_t[data_t.idioma == 'es']
data_t['idioma'].value_counts()

idioma
es    7794
Name: count, dtype: int64

Como se menciono en el entendimiento de datos, nos vamos a quedar con solo las reseñas que esten en español. Podemos observar que el dataframe ahora solo contiene reseñas en español

#### 2.1.3 Transformar los datos

In [17]:
stop_words = stopwords.words("spanish")
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words
def to_lowercase(words):
    new_words = []
    for word in words:
        new_words.append(word.lower())
    return new_words
    """Convert all characters to lowercase from list of tokenized words"""
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words
def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words
def remove_stopwords(words):
    stop_words = set(stopwords.words("spanish"))
    new_words = [word for word in words if word not in stop_words]
    return new_words
def preprocessing(words):
    words = to_lowercase(words)
    words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    return words
preprocessing()